In [1]:
import operator
import random

import numpy

from deap import base
from deap import benchmarks
from deap import creator
from deap import tools

C:\Users\Péricles\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


In [2]:
creator.create("Fitness", base.Fitness, weights=(1.0,))
creator.create("Particle", list, fitness=creator.Fitness, speed=list, smin=None, smax=None, best=None)

In [3]:
def generate(size, pmin, pmax, smin, smax):
    part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size))
    
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    
    part.smin = smin
    part.smax = smax
    return part

In [4]:
toolbox = base.Toolbox()

pmin = -100
pmax = 100

toolbox.register("particle", generate, size=10, pmin=pmin, pmax=pmax, smin=pmin*0.1, smax=pmax*0.1)
toolbox.register("population", tools.initRepeat, list, toolbox.particle)

In [5]:
def updateParticle(part, best, c1, c2):
    u1 = (random.uniform(0, c1) for _ in range(len(part)))
    u2 = (random.uniform(0, c2) for _ in range(len(part)))
    
    v_u1 = map(operator.mul, u1, map(operator.sub, part.best, part))
    v_u2 = map(operator.mul, u2, map(operator.sub, best, part))
    
    part.speed = map(operator.add, part.speed, map(operator.add, v_u1, v_u2))
    
    inertia_weights = (random.uniform(0.4, 0.9) for _ in range(len(part)))
    
    part.speed = list(map(operator.mul, inertia_weights, part.speed))
    
    for i, speed in enumerate(part.speed):
        if speed < part.smin:
            part.speed[i] = part.smin
        elif speed > part.smax:
            part.speed[i] = part.smax
            
    part[:] = list(map(operator.add, part, part.speed))

In [6]:
toolbox.register("update", updateParticle, c1=2.0, c2=2.0)
toolbox.register("evaluate", benchmarks.h1)

In [7]:
def main():
    pop = toolbox.population(n=30)
    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = ["gen", "evals"] + stats.fields

    GEN = 10000
    best = None
    
    all_bests = []

    for g in range(GEN):
        for part in pop:
            part.fitness.values = toolbox.evaluate(part)
            
            if not part.best or part.best.fitness < part.fitness:
                part.best = creator.Particle(part)
                part.best.fitness.values = part.fitness.values
                
            if not best or best.fitness < part.fitness:
                best = creator.Particle(part)
                best.fitness.values = part.fitness.values
        
        all_bests.append(best.fitness.values)
        
        for part in pop:
            toolbox.update(part, best)

        # Gather all the fitnesses in one list and print the stats
        logbook.record(gen=g, evals=len(pop), **stats.compile(pop))
        print(logbook.stream)
    
    return pop, logbook, best, all_bests

In [8]:
if __name__ == "__main__":
    results = main()
    
    for item in results[3]:
        print(item) 

gen	evals	avg      	std      	min       	max     
0  	30   	0.0225355	0.0446293	0.00077209	0.259228
1  	30   	0.0194548	0.0164687	0.000411181	0.072869
2  	30   	0.0338158	0.0441677	0.000756302	0.243489
3  	30   	0.0492376	0.0773725	0.00310245 	0.420219
4  	30   	0.0627097	0.0937355	0.00108579 	0.49452 
5  	30   	0.0879252	0.0889375	0.0153031  	0.383955
6  	30   	0.112738 	0.125673 	0.00112368 	0.485533
7  	30   	0.10651  	0.108386 	0.00428138 	0.475877
8  	30   	0.177787 	0.244862 	0.00604887 	1.33213 
9  	30   	0.236832 	0.188318 	0.0160557  	0.801674
10 	30   	0.229602 	0.187831 	0.0136378  	0.879723
11 	30   	0.361603 	0.339475 	0.00398663 	1.55664 
12 	30   	0.263632 	0.210388 	0.0112749  	0.920969
13 	30   	0.469643 	0.350337 	0.00840108 	1.35595 
14 	30   	0.389556 	0.285864 	0.033747   	1.2758  
15 	30   	0.61519  	0.465026 	0.0862586  	1.82914 
16 	30   	0.691125 	0.438845 	0.105169   	1.71614 
17 	30   	0.691221 	0.371928 	0.160895   	1.42246 
18 	30   	0.909164 	0.585787 	0.1

243	30   	2        	9.55216e-13	2          	2       
244	30   	2        	1.76471e-11	2          	2       
245	30   	2        	1.45397e-11	2          	2       
246	30   	2        	3.83014e-12	2          	2       
247	30   	2        	4.04388e-12	2          	2       
248	30   	2        	4.29139e-13	2          	2       
249	30   	2        	1.85716e-12	2          	2       
250	30   	2        	1.30922e-12	2          	2       
251	30   	2        	7.19442e-13	2          	2       
252	30   	2        	1.45251e-12	2          	2       
253	30   	2        	8.0114e-13 	2          	2       
254	30   	2        	5.14819e-13	2          	2       
255	30   	2        	7.43235e-13	2          	2       
256	30   	2        	4.68838e-13	2          	2       
257	30   	2        	9.97347e-13	2          	2       
258	30   	2        	9.69793e-13	2          	2       
259	30   	2        	4.80787e-13	2          	2       
260	30   	2        	3.30747e-14	2          	2       
261	30   	2        	2.942e-13  	2          	2 

404	30   	2        	4.44089e-16	2          	2       
405	30   	2        	4.44089e-16	2          	2       
406	30   	2        	4.44089e-16	2          	2       
407	30   	2        	4.44089e-16	2          	2       
408	30   	2        	4.44089e-16	2          	2       
409	30   	2        	4.44089e-16	2          	2       
410	30   	2        	4.44089e-16	2          	2       
411	30   	2        	4.44089e-16	2          	2       
412	30   	2        	4.44089e-16	2          	2       
413	30   	2        	4.44089e-16	2          	2       
414	30   	2        	4.44089e-16	2          	2       
415	30   	2        	4.44089e-16	2          	2       
416	30   	2        	4.44089e-16	2          	2       
417	30   	2        	4.44089e-16	2          	2       
418	30   	2        	4.44089e-16	2          	2       
419	30   	2        	4.44089e-16	2          	2       
420	30   	2        	4.44089e-16	2          	2       
421	30   	2        	4.44089e-16	2          	2       
422	30   	2        	4.44089e-16	2          	2 

671	30   	2        	4.44089e-16	2          	2       
672	30   	2        	4.44089e-16	2          	2       
673	30   	2        	4.44089e-16	2          	2       
674	30   	2        	4.44089e-16	2          	2       
675	30   	2        	4.44089e-16	2          	2       
676	30   	2        	4.44089e-16	2          	2       
677	30   	2        	4.44089e-16	2          	2       
678	30   	2        	4.44089e-16	2          	2       
679	30   	2        	4.44089e-16	2          	2       
680	30   	2        	4.44089e-16	2          	2       
681	30   	2        	4.44089e-16	2          	2       
682	30   	2        	4.44089e-16	2          	2       
683	30   	2        	4.44089e-16	2          	2       
684	30   	2        	4.44089e-16	2          	2       
685	30   	2        	4.44089e-16	2          	2       
686	30   	2        	4.44089e-16	2          	2       
687	30   	2        	4.44089e-16	2          	2       
688	30   	2        	4.44089e-16	2          	2       
689	30   	2        	4.44089e-16	2          	2 

943	30   	2        	4.44089e-16	2          	2       
944	30   	2        	4.44089e-16	2          	2       
945	30   	2        	4.44089e-16	2          	2       
946	30   	2        	4.44089e-16	2          	2       
947	30   	2        	4.44089e-16	2          	2       
948	30   	2        	4.44089e-16	2          	2       
949	30   	2        	4.44089e-16	2          	2       
950	30   	2        	4.44089e-16	2          	2       
951	30   	2        	4.44089e-16	2          	2       
952	30   	2        	4.44089e-16	2          	2       
953	30   	2        	4.44089e-16	2          	2       
954	30   	2        	4.44089e-16	2          	2       
955	30   	2        	4.44089e-16	2          	2       
956	30   	2        	4.44089e-16	2          	2       
957	30   	2        	4.44089e-16	2          	2       
958	30   	2        	4.44089e-16	2          	2       
959	30   	2        	4.44089e-16	2          	2       
960	30   	2        	4.44089e-16	2          	2       
961	30   	2        	4.44089e-16	2          	2 

...

(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)


(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)


(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)


(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)


(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
(1.99999999992158,)
